In [1]:

import sys

sys.path.append("..")
from data.database import SessionLocal
from data.data_models import Products

from tqdm import tqdm

In [7]:

import os
import requests
from openai import OpenAI
from sqlalchemy.orm import Session
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")

client = OpenAI()
def generate_and_save_image(prompt, filename):
    # Check if the image already exists
    if not os.path.exists(filename):
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )

        image_url = response.data[0].url
        image_response = requests.get(image_url)
        
        # Ensure the directory exists
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        with open(filename, 'wb') as f:
            f.write(image_response.content)
    else:
        print(f"Image already exists: {filename}")

def process_restaurants(db: Session):
    products = db.query(Products).all()
    print("Generating restaurant images...")
    print(products)
    for product in tqdm(products):
        try:
            prompt = f"A high-quality, realistic image thumbnail of a {product.name} by {product.brand}. The product is displayed on a clean, white background with soft shadows to make it stand out. The image is crisp and clear, showcasing the product's details, texture, and branding elements. This image will be used as a thumbnail for an online store, so the product should be centered, well-lit, and visually appealing."
            filename = os.path.join('../../../frontend/src/assets/', product.image)
            generate_and_save_image(prompt, filename)
        except Exception as e:
            print(e)


# Get the database session
db = SessionLocal()

try:
    # Process restaurants and foods
    process_restaurants(db)
except Exception as e:
    print(e)

finally:
    # Close the session
    db.close()

Generating restaurant images...
[<data.data_models.Products object at 0x000001F39B1BA0A0>, <data.data_models.Products object at 0x000001F39B1BA130>, <data.data_models.Products object at 0x000001F39B1BA040>, <data.data_models.Products object at 0x000001F39B1C40A0>, <data.data_models.Products object at 0x000001F39B1C4100>, <data.data_models.Products object at 0x000001F39B1C4160>, <data.data_models.Products object at 0x000001F39B1C41C0>, <data.data_models.Products object at 0x000001F39B1C4220>, <data.data_models.Products object at 0x000001F39B1C4280>, <data.data_models.Products object at 0x000001F39B1C42E0>, <data.data_models.Products object at 0x000001F39B1C4340>, <data.data_models.Products object at 0x000001F39B1C43A0>, <data.data_models.Products object at 0x000001F39B1C4400>, <data.data_models.Products object at 0x000001F39B1C4460>, <data.data_models.Products object at 0x000001F39B1C44C0>, <data.data_models.Products object at 0x000001F39B1C4520>, <data.data_models.Products object at 0x

 24%|██▍       | 6/25 [02:07<06:44, 21.26s/it]


KeyboardInterrupt: 